In [5]:
import instant
import weather
# from weather import Weather
dir(instant)

['Instant',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'cast_count_all',
 'find_legit',
 'load_legit',
 'sweep',
 'test_load_legit']

In [6]:
# import Extract
dir(Extract)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__']

In [8]:
# from Extract import *
# print(dir())
from Extract.

['Extract', 'In', 'Out', '_', '_1', '_5', '_6', '_7', '__', '___', '__builtin__', '__builtins__', '__doc__', '__loader__', '__name__', '__package__', '__spec__', '_dh', '_i', '_i1', '_i2', '_i3', '_i4', '_i5', '_i6', '_i7', '_i8', '_ih', '_ii', '_iii', '_oh', 'exit', 'get_ipython', 'quit']


In [5]:
''' Defines the Weather class and related functions. '''

import time
import json

from pyowm import OWM
from pyowm.weatherapi25.forecast import Forecast
from pyowm.exceptions.api_response_error import NotFoundError
from pyowm.exceptions.api_call_error import APICallTimeoutError
from pyowm.exceptions.api_call_error import APIInvalidSSLCertificateError

from config import OWM_API_key_loohoo as loohoo_key
from config import OWM_API_key_masta as masta_key


class Weather:
    ''' A dictionary of weather variables and their observed/forecasted values
    for a given instant in time at a specified location.
    '''
    
    def __init__(self, location, _type, data=None):
        '''
        :param location: can be either valid US zipcode or coordinate dictionary
        :type location: If this param is a zipcode, it should be str, otherwise
        dict
        :param _type: Indicates whether its data is observational or forecasted
        :type _type: string  It must be either 'observation' or 'forecast'
        '''

        self.type = _type
        self.loc = location
        self.weather = data
        # make the _id for each weather according to its reference time
        if _type == 'forecast' and 'reference_time' in data:
            self._id = f'{str(location)}{str(data["reference_time"])}'
        elif _type == 'observation' and 'reference_time' in data:
            self._id = f'''
            {str(location)}{str(10800 * (data["reference_time"]//10800 + 1))}'''
        self.as_dict = {'_id': self._id,
                       '_type': self.type,
                        'weather': self.weather
                       }

    def to_inst(self):
        ''' This will find the id'd Instant and add the Weather to it according 
        to its type. '''
        
        from instant import Instant
        from get_and_make import load_instants_from_db
        
        if self.type == 'observation':
            _id = self._id
            instants.setdefault(_id, Instant(_id, observations=self.weather))
            return
        if self.type == 'forecast':
            _id = self._id
            instants[_id]['forecasts'].append(weather)
            return


def get_data_from_weather_api(owm, location):
    ''' Makes api calls for observations and forecasts and handles the API call
    errors.

    :param owm: the OWM API object
    :type owm: pyowm.OWM
    :param location: the coordinates or zipcode reference for the API call.
    :type location: if location is a zipcode, then type is a string;
    if location is a coordinates, then tuple or dict.

    returns: the API data
    '''
    
    result = None
    tries = 1
    while result is None and tries < 4:
        try:
            if type(location) == dict:
                result = owm.three_hours_forecast_at_coords(**location)
                return result
            elif type(location) == str:
                result = owm.weather_at_zip_code(location, 'us')
                return result
        except APIInvalidSSLCertificateError:
            loc = zipcode or 'lat: {}, lon: {}'.format(coords['lat'], coords['lon'])
            print(f'SSL error with {loc} on attempt {tries} ...trying again')
            if type(location) == dict:
                owm_loohoo = OWM(loohoo_key)
                owm = owm_loohoo
            elif type(location) == str:
                owm_masta = OWM(masta_key)
                owm = owm_masta
        except APICallTimeoutError:
            loc = location[:] or 'lat: {}, lon: {}'.format(location['lat'],
                                                           location['lon'])
            print(f'''Timeout error with {loc} on attempt {tries}... waiting 1
                  second then trying again''')
            time.sleep(1)
        tries += 1
    if tries == 4:
        print('''tried 3 times without response; breaking out and causing an
        error that will crash your current colleciton process...fix that!''')
        return ### sometime write something to keep track of the zip and
                ### instant that isn't collected ###

def get_current_weather(location):
    ''' Get the current weather for the given zipcode or coordinates.

    :param location: the coordinates or zipcode reference for the API call.
    :type location: if location is a zipcode, then type is a string;
    if location is a coordinates, then tuple or dict.

    :return: the raw weather object
    :type: json
    '''
    owm = OWM(loohoo_key)

    m = 0
    # Try several times to get complete the API request
    while m < 4:
        try:
            # get the raw data from the OWM and make a Weather from it
            result = get_data_from_weather_api(owm, location)
            result = json.loads(result.to_JSON())  # the current weather for the given zipcode
            weather = Weather(location, 'observation', result['Weather'])
            return weather
        except APICallTimeoutError:
            owm = owm_loohoo
            m += 1
    print(f'Did not get current weather for {location} and reset owm')
    return
    
def five_day(location):
    ''' Get each weather forecast for the corrosponding coordinates
    
    :param coords: the latitude and longitude for which that that weather is
    being forecasted
    :type coords: tuple containing the latitude and logitude for the forecast

    :return casts: the five day, every three hours, forecast for the zip code
    :type casts: dict
    '''

    owm = OWM(masta_key)

    Forecast = get_data_from_weather_api(owm, location).get_forecast()
    forecast = json.loads(Forecast.to_JSON())
    casts = [] # This is for the weather objects created in the for loop below.
    for data in forecast['weathers']:
        # Make an _id for the next Weather to be created, create the weather, 
        # append it to the casts list.
        instant = data['reference_time']
        data['_id'] = f'{str(location)}{str(instant)}'
        casts.append(Weather(location, 'forecast', data))
    return casts


In [6]:
dir()
# OWM_API_key_masta = 'ec7a9ff0f4a568d9e8e6ef8b810c599e'
# OWM_API_key_loohoo ='ccf670fd173f90d5ae9c84ef6372573d'

# weather = get_current_weather('27006')

['APICallTimeoutError',
 'APIInvalidSSLCertificateError',
 'Forecast',
 'In',
 'NotFoundError',
 'OWM',
 'Out',
 'Weather',
 '_',
 '_1',
 '_4',
 '__',
 '___',
 '__builtin__',
 '__builtins__',
 '__doc__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '_dh',
 '_i',
 '_i1',
 '_i2',
 '_i3',
 '_i4',
 '_i5',
 '_i6',
 '_ih',
 '_ii',
 '_iii',
 '_oh',
 'exit',
 'five_day',
 'get_current_weather',
 'get_data_from_weather_api',
 'get_ipython',
 'json',
 'loohoo_key',
 'masta_key',
 'quit',
 'time',
 'weather']

In [63]:
from Extract.make_instants import find_data
# set database and collection for testing
# database = 'test'
collection = 'instant_temp'
# create a dict to hold the instants pulled from the database
instants = {}
data = find_data(client, database, collection)
# add each doc to instants and set its key and _id to the same values
for item in data:
    instants[f'{item["_id"]}'] = item['_id']
print(len(instants))

78706


In [64]:
weather.to_inst()

In [21]:
len(instants)

78707

In [58]:
weather = get_current_weather('27006')

In [59]:
weather.to_inst()

In [60]:
len(instants)
# instants[weather._id].as_dict

78708

In [69]:
dir()

NameError: name 'Extract' is not defined